# Análisis con XGBoost

¿MODIFICAR LOS ENFOQUES? y los títulos

En este notebook se encuentran dos enfoques de análisis: en un enfoque (por estudiante) los experimentos consideran que cada observación es la información de un estudiante en un cuatrimestre (el mismo estudiante puede aparecer en varios cuatrimestres); mientras que en el otro enfoque (por curso) las observaciones son los datos agrupados por curso.  

XGBoost puede encargarse internamente de las variables categóricas sin necesidad de hacer one-hot encoding. Debido a esto se realizaron análisis dejando que XGBoost trate a las variables categóricas como tales, pero también se probaron versiones con one-hot encoding de algunas variables para resaltar algunas características.

**Tabla de contenidos**<a id='toc0_'></a>    
- 1. [Importación de librerías y datos](#toc1_)    
- 2. [Abandona entre primer y segundo parcial](#toc2_)    
  - 2.1. [Modelo base](#toc2_1_)    
  - 2.2. [Test de permutaciones para importancias](#toc2_2_)    
  - 2.3. [Importancias mediante eliminación](#toc2_3_)    
  - 2.4. [Repeticiones con diferentes semillas](#toc2_4_)    
  - 2.5. [Codificación one-hot](#toc2_5_)    
- 3. [Estudiantes con notas altas](#toc3_)    
  - 3.1. [Modelo con SEDE y pa1](#toc3_1_)    
  - 3.2. [Modelo con sala y pa1](#toc3_2_)    
  - 3.3. [Codificación one-hot: SEDE](#toc3_3_)    
  - 3.4. [Modelo sin pa1](#toc3_4_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## 1. <a id='toc1_'></a>[Importación de librerías y datos](#toc0_)

In [31]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import average_precision_score
import xgboost as xgb
import importlib

# Funciones para entrenar y evaluar los modelos.
import lib_entrenar as le
 
import matplotlib.pyplot as plt

# Descomentar lo siguiente para generar gráficos pgf:

# %matplotlib inline
# import matplotlib
# matplotlib.use("pgf")
# matplotlib.rcParams.update({
#     "pgf.texsystem": "pdflatex",
#     'font.family': 'serif',
#     'text.usetex': True,
#     'pgf.rcfonts': False,
# })

In [32]:
# Necesario cuando se realizan modificaciones en el módulo.
# importlib.reload(le)

In [33]:
df = pd.read_csv("../datos/dataset_02-feateng.csv")
df['SEDE'] = df['SEDE'].astype('str')
df['MATERIA'] = df['MATERIA'].astype('str')
df['edad'] = df['edad'].astype('category')
print("Cantidad de observaciones: ",len(df),"\n")
print("Variables: ",df.columns.values)
df['objetivo'] = (df['condición'] == 'Abandona2').astype(int)
df2 = df.loc[(df['valido2']==1) & (df['condición'] != 'Abandona1')].copy()

print("\nCantidad de observaciones válidas: ",len(df2))
print("Cantidad de cursos:", len(df2['curso'].unique()))
print("Positivos/Total: ", round(df2.objetivo.sum()/len(df2),3))

Cantidad de observaciones:  233615 

Variables:  ['anio' 'cuat' 'SEDE' 'MATERIA' 'pa1' 'pa2' 'Final' 'codCarrera'
 'facultad' 'rem1' 'rem2' 'estudiante' 'extranjero' 'curso' 'turno'
 'n_alum' 'p_ext' 'recurso' 'p_recursa' 'sala' 'pa1_prom' 'pa2_prom'
 'final_prom' 'edad' 'prom_edad' 'condición' 'abandona1_p' 'abandona2_p'
 'valido1' 'valido2']

Cantidad de observaciones válidas:  106987
Cantidad de cursos: 1763
Positivos/Total:  0.24


## 2. <a id='toc2_'></a>[Abandona entre primer y segundo parcial](#toc0_)


Se seleccionan las observaciones válidas para este análisis, y todas las variables consistentes.

In [34]:
df['objetivo'] = (df['condición'] == 'Abandona2').astype(int)
df2 = df.loc[(df['valido2']==1) & (df['condición'] != 'Abandona1')].copy()

print("Cantidad de observaciones válidas: ",len(df2))
print("Positivos/Total: ", round(df2.objetivo.sum()/len(df2),3))

Cantidad de observaciones válidas:  106987
Positivos/Total:  0.24


Separar un 30% para conjunto de test y guardar los índices para utilizar los mismos conjuntos en otros scripts.  

In [35]:
# No ejecutar esta celda.
# Este código fue utilizado para generar los índices de los
# conjuntos de entrenamiento y test para algunos experimentos. 

# columnas = ['cuat', 'SEDE', 'MATERIA', 'pa1', 'pa1_prom', 'codCarrera', 'facultad',
#         'extranjero', 'edad', 'prom_edad', 'turno', 'n_alum', 'p_ext',
#         'recurso', 'p_recursa', 'sala', 'abandona1_p']
# X = df2[columnas].copy()
# y = df2[['objetivo']].copy()
# print("Variables: ", X.columns.values)

# # Separar los datos.
# X_train, X_test, y_train, y_test = train_test_split(X, y,
#         test_size=0.3,
#         random_state=1,
#         stratify=y)

# # Guardar los índices.
# selected = X_train.index.to_numpy()
# np.savetxt("indices_estudiante_train.txt.gz", selected)
# selected = X_test.index.to_numpy()
# np.savetxt("indices_estudiante_test.txt.gz", selected)

### 2.1. <a id='toc2_1_'></a>[Modelo base](#toc0_)

``sala`` pertenece a cada ``SEDE``, por lo cual ``SEDE`` puede ser considerado como una recategorización de ``sala``. Debido a esto se realizan dos pruebas con una y otra variable.  

Con la variable ``SEDE`` y **no ``sala``**, el modelo obtenido es el resultado del script <code>01_base.py</code>.

In [43]:
bst = xgb.Booster(model_file="modelo_01_base.json")
importancia = pd.DataFrame.from_dict(bst.get_score(importance_type='total_gain'), orient='index')
importancia = importancia.sort_values(by=0,ascending=False)
importancia[0] = round(importancia[0]/importancia[0].min(),2)
importancia.columns = ['importancia']
importancia

,importancia
pa1,168.75
abandona1_p,12.62
pa1_prom,10.18
prom_edad,9.60
p_ext,8.76
p_recursa,8.50
n_alum,7.60
facultad,3.20
turno,2.91
edad,2.89


La importancia de pa1 es dominante.

In [37]:
conj_test = np.loadtxt("indices_estudiante_test.txt.gz")
df2_test = df.loc[conj_test].copy()
print("Cantidad de observaciones para test: ",len(df2_test))
print("Positivos/Total: ", round(df2_test.objetivo.sum()/len(df2_test),3))
columnas = ['cuat', 'SEDE', 'MATERIA', 'pa1', 'pa1_prom', 'facultad',
        'extranjero', 'edad', 'prom_edad', 'turno', 'n_alum', 'p_ext',
        'recurso', 'p_recursa', 'abandona1_p']
X_test = df2_test[columnas].copy()
y_test = df2_test[['objetivo']].copy()
cats = ['cuat', 'SEDE', 'MATERIA', 'facultad', 'extranjero', 'edad',
       'turno']
for col in cats:
   X_test[col] = X_test[col].astype('category')

dtest = xgb.DMatrix(X_test, y_test, enable_categorical=True)

y_pred = bst.predict(dtest)
print("Average precision score:", round(average_precision_score(y_test, y_pred),3))

Cantidad de observaciones para test:  32097
Positivos/Total:  0.24
Average precision score: 0.587


El AP score está alejado de un clasificador al azar.

### 2.2. <a id='toc2_2_'></a>[Test de permutaciones para importancias](#toc0_)

Se analizan las importancias mediante el test de permutaciones.  


In [47]:
bst = xgb.Booster(model_file="modelo_01_base.json")
conj_test = np.loadtxt("indices_estudiante_test.txt.gz")
df2_test = df.loc[conj_test].copy()
columnas = ['cuat', 'SEDE', 'MATERIA', 'pa1', 'pa1_prom', 'facultad',
        'extranjero', 'edad', 'prom_edad', 'turno', 'n_alum', 'p_ext',
        'recurso', 'p_recursa', 'abandona1_p']
X_test = df2_test[columnas].copy()
y_test = df2_test[['objetivo']].copy()
cats = ['cuat', 'SEDE', 'MATERIA', 'facultad', 'extranjero', 'edad',
       'turno']
for col in cats:
   X_test[col] = X_test[col].astype('category')

X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

dtest = xgb.DMatrix(X_test, y_test, enable_categorical=True)
y_pred = bst.predict(dtest)
base = average_precision_score(y_test, y_pred)
print("Base =", base)

importancias = np.array([])
for col in columnas:
    Xsave = X_test[col].copy()
    X_test[col] = np.random.permutation(X_test[col])
    for c in cats:
        X_test[c] = X_test[c].astype('category')
    dtest = xgb.DMatrix(X_test, y_test, enable_categorical=True)
    y_pred = bst.predict(dtest)
    aps = average_precision_score(y_test, y_pred)
    importancias = np.concatenate([importancias, [round(base - aps,3)]])
    X_test[col] = Xsave
df_importancias = pd.DataFrame({'variable':columnas, 'importancia':importancias})
df_importancias.sort_values(by='importancia',ascending=False).reset_index(drop=True)

Base = 0.5872546996544572


,variable,importancia
0,pa1,0.314
1,abandona1_p,0.020
2,edad,0.016
3,p_recursa,0.007
4,pa1_prom,0.006
5,turno,0.005
6,facultad,0.004
7,prom_edad,0.004
8,extranjero,0.003
9,SEDE,0.002


### 2.3. <a id='toc2_3_'></a>[Importancias mediante eliminación](#toc0_)

Se eliminan las variables de a una y se reentrenan los modelos.

Resultados del script <code>01_dropout.py</code>

In [52]:
aps = np.loadtxt("salida_01_droput_aps.txt")
importancias = base - aps
variables = ['cuat', 'SEDE', 'MATERIA', 'pa1', 'pa1_prom', 'facultad',
        'extranjero', 'edad', 'prom_edad', 'turno', 'n_alum', 'p_ext',
        'recurso', 'p_recursa', 'abandona1_p']
print(aps)
importancias


[0.58786242 0.58965033 0.58701632 0.35849124 0.58845474 0.58573559
 0.59022204 0.57403865 0.58818321 0.58513774 0.5888438  0.5855067
 0.59173659 0.58725516 0.58367574]


array([-6.07720895e-04, -2.39563169e-03,  2.38380712e-04,  2.28763459e-01,
       -1.20003921e-03,  1.51910849e-03, -2.96734408e-03,  1.32160487e-02,
       -9.28510147e-04,  2.11696087e-03, -1.58910405e-03,  1.74799764e-03,
       -4.48189279e-03, -4.61050126e-07,  3.57896175e-03])

### 2.4. <a id='toc2_4_'></a>[Repeticiones con diferentes semillas](#toc0_)

Se repite el entrenamiento 10 veces, con diferentes conjuntos de entrenamiento y prueba, para generar estadística y al mismo tiempo verificar si el orden de importancia de las variables cambia según los ejemplos que se utilicen para el entrenamiento.  

En cada experimento también se seleccionan distintas variables.

In [25]:
aps = np.loadtxt("salida_02a_average_precision_scores.txt")
print("Average precision score:")
print(round(aps.mean(),3),"+/-",round(aps.std(),3))

Average precision score:
0.588 +/- 0.005


In [26]:
importancias = pd.read_csv("salida_02a_importancias.csv")
for i in range(importancias.shape[1] - 1):
    importancias[str(i)] = round(importancias[str(i)]/importancias[str(i)].min(),1)
columnas = np.array(['variable'])
for i in range(importancias.shape[1] - 1):
    columnas = np.concatenate([columnas,[str(i+1)]])
importancias.columns = columnas
importancias.sort_values(by='1',ascending=False, ignore_index=True)

,variable,1,2,3,4,5,6,7,8,9,10
0,pa1,521.8,209.4,217.9,210.0,568.7,351.0,192.5,190.1,152.8,175.6
1,abandona1_p,22.5,9.6,9.6,10.4,18.4,15.9,11.1,12.5,10.3,8.3
2,pa1_prom,12.5,6.4,5.9,5.5,4.4,9.7,8.3,8.2,8.9,6.4
3,prom_edad,11.8,6.9,6.8,5.5,6.8,9.9,8.5,8.7,8.4,5.4
4,p_recursa,9.8,5.8,5.3,6.0,8.0,9.3,6.7,8.1,7.3,5.9
5,p_ext,9.7,4.9,3.8,4.6,2.9,7.8,6.5,6.3,7.5,4.0
6,n_alum,7.8,4.3,4.1,4.5,3.9,5.9,5.9,5.5,6.6,4.4
7,edad,7.4,3.1,3.7,3.8,7.7,6.6,4.0,3.7,3.1,3.1
8,codCarrera,7.3,3.8,3.2,3.2,5.6,6.4,3.8,5.5,5.5,3.2
9,turno,6.3,3.5,3.0,2.9,7.8,5.2,3.3,3.1,2.7,2.8


In [27]:
importancias_rank = importancias.copy()
for i in range(importancias.shape[1] - 1):
    importancias_rank[str(i+1)] = importancias_rank[str(i+1)].rank()
importancias_rank.sort_values(by='1',ascending=False, ignore_index=True)

,variable,1,2,3,4,5,6,7,8,9,10
0,pa1,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0,16.0
1,abandona1_p,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0,15.0
2,pa1_prom,14.0,13.0,13.0,12.5,9.0,13.0,13.0,13.0,14.0,14.0
3,prom_edad,13.0,14.0,14.0,12.5,11.0,14.0,14.0,14.0,13.0,12.0
4,p_recursa,12.0,12.0,12.0,14.0,14.0,12.0,12.0,12.0,11.0,13.0
5,p_ext,11.0,11.0,10.0,11.0,7.0,11.0,11.0,11.0,12.0,10.0
6,n_alum,10.0,10.0,11.0,10.0,8.0,8.0,10.0,9.5,10.0,11.0
7,edad,9.0,7.0,9.0,9.0,12.0,10.0,9.0,8.0,8.0,8.0
8,codCarrera,8.0,9.0,8.0,8.0,10.0,9.0,8.0,9.5,9.0,9.0
9,turno,7.0,8.0,7.0,7.0,13.0,7.0,7.0,7.0,7.0,7.0


In [28]:
df_top5 = pd.DataFrame()
top = np.array([])
for i in range(10):
    top = np.concatenate([top, importancias.sort_values(by=str(i+1),ascending=False, ignore_index=True)['variable'].values[0:5]])
df_top5['variable'] = np.unique(top)
df_top5 = df_top5.sort_values(by='variable',ascending=True, ignore_index=True)
for i in range(10):
    top = importancias.sort_values(by=str(i+1),ascending=False, ignore_index=True)['variable'].values[0:5]
    df_top5[str(i+1)] = ''
    for j in range(len(top)):
        df_top5.loc[df_top5['variable'] == top[j],str(i+1)] = str(j+1)
df_top5

,variable,1,2,3,4,5,6,7,8,9,10
0,abandona1_p,2,2,2,2,2,2,2,2,2,2
1,edad,,,,,5,,,,,
2,p_ext,,,,,,,,,5,
3,p_recursa,5,5,5,3,3,5,5,5,,4
4,pa1,1,1,1,1,1,1,1,1,1,1
5,pa1_prom,3,4,4,4,,4,4,4,3,3
6,prom_edad,4,3,3,5,,3,3,3,4,5
7,turno,,,,,4,,,,,


In [30]:
print(df_top5)

      variable  1  2  3  4  5  6  7  8  9 10
0  abandona1_p  2  2  2  2  2  2  2  2  2  2
1         edad              5               
2        p_ext                          5   
3    p_recursa  5  5  5  3  3  5  5  5     4
4          pa1  1  1  1  1  1  1  1  1  1  1
5     pa1_prom  3  4  4  4     4  4  4  3  3
6    prom_edad  4  3  3  5     3  3  3  4  5
7        turno              4               


## 3. <a id='toc3_'></a>[Estudiantes con notas altas](#toc0_)

Se restringe el análisis a las observaciones con calificaciones en el parcial 1 mayores a 3.

In [38]:
df_altas = df.loc[df['pa1'] > 3].copy()
df_altas = df_altas.loc[(df_altas['valido2']==1) & (df_altas['condición'] != 'Abandona1')].copy()
df_altas['objetivo'] = df_altas['objetivo'].astype('int')
df_altas = df_altas.reset_index(drop=True)
print("\nCantidad de observaciones con notas > 3: ", len(df_altas))
print("Positivos/Total: ", round(df_altas.objetivo.sum()/len(df_altas),3))


Cantidad de observaciones con notas > 3:  51789
Positivos/Total:  0.042


Separar un 30% para conjunto de test y guardar los índices para utilizar los mismos conjuntos en otros scripts.

In [ ]:
# No ejecutar esta celda.
# Este código fue utilizado para generar los índices de los
# conjuntos de entrenamiento y test para algunos experimentos. 
# df_altas = df_altas[['cuat', 'SEDE', 'MATERIA', 'pa1', 'codCarrera',
#         'facultad', 'extranjero', 'turno', 'n_alum', 'p_ext', 'recurso',
#         'p_recursa', 'sala', 'pa1_prom', 'edad', 'prom_edad',
#         'abandona1_p','objetivo']]
# df_altas.to_csv('../datos/dataset_30-notas_altas.csv', index=False)

# X = df_altas.drop(['objetivo'], axis=1).copy()
# y = df_altas[['objetivo']].copy()

# # Separar los datos.
# X_train, X_test, y_train, y_test = train_test_split(X, y,
#         test_size=0.3,
#         random_state=1,
#         stratify=y)

# # Guardar los índices.
# selected = X_train.index.to_numpy()
# np.savetxt("indices_notas_altas_train.txt.gz", selected)
# selected = X_test.index.to_numpy()
# np.savetxt("indices_notas_altas_test.txt.gz", selected)

### 3.1. <a id='toc3_1_'></a>[Modelo con SEDE y pa1](#toc0_)

Con la variable ``SEDE`` y **no ``sala``**, el modelo obtenido es el resultado del script <code>30_altas.py</code>.

In [42]:
bst = xgb.Booster(model_file="modelo_30_altas_aucpr.json")
importancia = pd.DataFrame.from_dict(bst.get_score(importance_type='total_gain'), orient='index')
importancia = importancia.sort_values(by=0,ascending=False)
importancia[0] = importancia[0]/importancia[0].min()
importancia.columns = ['importancia']
importancia

,importancia
pa1,8.727299
abandona1_p,7.560513
pa1_prom,6.460779
prom_edad,6.214392
p_recursa,5.515126
p_ext,5.448417
n_alum,4.604799
SEDE,2.369530
MATERIA,2.038714
edad,1.907328


In [40]:
conj_test = np.loadtxt("indices_notas_altas_test.txt.gz")
df2_test = df_altas.loc[conj_test].copy()
print("Cantidad de observaciones para test: ",len(df2_test))
print("Positivos/Total: ", round(df2_test.objetivo.sum()/len(df2_test),3))
columnas = ['cuat', 'SEDE', 'MATERIA', 'pa1', 'facultad',
       'extranjero', 'turno', 'n_alum', 'p_ext', 'recurso', 'p_recursa',
       'pa1_prom', 'edad', 'prom_edad', 'abandona1_p']
X_test = df2_test[columnas].copy()
y_test = df2_test[['objetivo']].copy()
cats = ['cuat', 'SEDE', 'MATERIA', 'facultad', 'extranjero', 'edad',
       'turno']
for col in cats:
   X_test[col] = X_test[col].astype('category')

dtest = xgb.DMatrix(X_test, y_test, enable_categorical=True)

y_pred = bst.predict(dtest)
print("Average precision score:", round(average_precision_score(y_test, y_pred),3))

Cantidad de observaciones para test:  15537
Positivos/Total:  0.042
Average precision score: 0.126


Es un resultado marginal, pero está por encima del aleatorio.

### 3.2. <a id='toc3_2_'></a>[Modelo con sala y pa1](#toc0_)

Con la variable ``sala`` y **no ``SEDE``**, el modelo obtenido es el resultado del script <code>31_altas.py</code>.

In [41]:
bst = xgb.Booster(model_file="modelo_31_altas_aucpr.json")
importancia = pd.DataFrame.from_dict(bst.get_score(importance_type='total_gain'), orient='index')
importancia = importancia.sort_values(by=0,ascending=False)
importancia[0] = importancia[0]/importancia[0].min()
importancia.columns = ['importancia']
importancia

,importancia
pa1,11.685932
abandona1_p,10.413203
prom_edad,8.993553
pa1_prom,7.716251
n_alum,6.850746
p_recursa,6.548778
p_ext,5.793159
sala,5.188944
facultad,2.868785
turno,2.647928


In [42]:
conj_test = np.loadtxt("indices_notas_altas_test.txt.gz")
df2_test = df_altas.loc[conj_test].copy()
print("Cantidad de observaciones para test: ",len(df2_test))
print("Positivos/Total: ", round(df2_test.objetivo.sum()/len(df2_test),3))
columnas = ['cuat', 'sala', 'MATERIA', 'pa1', 'facultad',
       'extranjero', 'turno', 'n_alum', 'p_ext', 'recurso', 'p_recursa',
       'pa1_prom', 'edad', 'prom_edad', 'abandona1_p']
X_test = df2_test[columnas].copy()
y_test = df2_test[['objetivo']].copy()
cats = ['cuat', 'sala', 'MATERIA', 'facultad', 'extranjero', 'edad',
       'turno']
for col in cats:
   X_test[col] = X_test[col].astype('category')

dtest = xgb.DMatrix(X_test, y_test, enable_categorical=True)

y_pred = bst.predict(dtest)
print("Average precision score:", round(average_precision_score(y_test, y_pred),3))

Cantidad de observaciones para test:  15537
Positivos/Total:  0.042
Average precision score: 0.132


### 3.3. <a id='toc3_3_'></a>[Codificación one-hot: SEDE](#toc0_)

Codificación one-hot de las variables categóricas. Con SEDE y no sala. El modelo obtenido es el resultado del script <code>32_altas.py</code>.

In [43]:
bst = xgb.Booster(model_file="modelo_32_altas_aucpr.json")
importancia = pd.DataFrame.from_dict(bst.get_score(importance_type='total_gain'), orient='index')
importancia = importancia.sort_values(by=0,ascending=False)
importancia[0] = importancia[0]/importancia[0].min()
importancia.columns = ['importancia']
importancia

,importancia
abandona1_p,1049.700209
pa1,1005.418571
prom_edad,933.778332
pa1_prom,829.358385
p_recursa,738.856722
n_alum,663.016564
p_ext,627.340724
edad_3,249.261086
recurso,220.080976
MATERIA,185.917601


### 3.4. <a id='toc3_4_'></a>[Modelo sin pa1](#toc0_)

Resultado del script <code>33_altas.py</code>.

In [44]:
bst = xgb.Booster(model_file="modelo_33_altas_aucpr.json")
importancia = pd.DataFrame.from_dict(bst.get_score(importance_type='total_gain'), orient='index')
importancia = importancia.sort_values(by=0,ascending=False)
importancia[0] = importancia[0]/importancia[0].min()
importancia.columns = ['importancia']
importancia

,importancia
abandona1_p,11.165182
prom_edad,11.009246
pa1_prom,9.440964
p_recursa,8.909869
p_ext,7.324520
n_alum,6.565862
SEDE,4.466109
turno,3.266483
facultad,3.176499
edad,3.104831


In [45]:
conj_test = np.loadtxt("indices_notas_altas_test.txt.gz")
df2_test = df_altas.loc[conj_test].copy()
print("Cantidad de observaciones para test: ",len(df2_test))
print("Positivos/Total: ", round(df2_test.objetivo.sum()/len(df2_test),3))
columnas = ['cuat', 'SEDE', 'MATERIA', 'facultad',
       'extranjero', 'turno', 'n_alum', 'p_ext', 'recurso', 'p_recursa',
       'pa1_prom', 'edad', 'prom_edad', 'abandona1_p']
X_test = df2_test[columnas].copy()
y_test = df2_test[['objetivo']].copy()
cats = ['cuat', 'SEDE', 'MATERIA', 'facultad', 'extranjero', 'edad',
       'turno']
for col in cats:
   X_test[col] = X_test[col].astype('category')

dtest = xgb.DMatrix(X_test, y_test, enable_categorical=True)

y_pred = bst.predict(dtest)
print("Average precision score:", round(average_precision_score(y_test, y_pred),3))

Cantidad de observaciones para test:  15537
Positivos/Total:  0.042
Average precision score: 0.108


El AP bajó poco, de acuerdo a la importancia informada por XGBoost.